### AASMANN (modified) is leveraging the following methods as mentioned below.




*   DPN (Dual Path Network)
*   EfficientZERO
*   MRD-CNN
*   FractalNet
*   Blockwise Learning Networks



1. DPN (DUAL PATH NETWORK)

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [39]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_data(image_directory):
    images = []
    labels = []
    class_names = os.listdir(image_directory)

    for label_idx, class_name in enumerate(class_names):
        class_path = os.path.join(image_directory, class_name)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            image = load_img(image_path, target_size=(224, 224))
            image = img_to_array(image)
            images.append(image)
            labels.append(label_idx)

    images = np.array(images)
    labels = np.array(labels)
    return images, labels

image_directory = "/content/drive/MyDrive/UCMerced_LandUse/UCMerced_LandUse/Images_converted"
X, y = load_data(image_directory)
X = X / 255.0

In [40]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Train data: {X_train.shape}, Validation data: {X_val.shape}, Test data: {X_test.shape}")


Train data: (1470, 224, 224, 3), Validation data: (315, 224, 224, 3), Test data: (315, 224, 224, 3)


In [41]:
def dpn_block(input_tensor, filters, kernel_size=3, stride=1, use_bias=False):
    path1 = layers.Conv2D(filters, kernel_size, strides=stride, padding='same', use_bias=use_bias)(input_tensor)
    path1 = layers.BatchNormalization()(path1)
    path1 = layers.ReLU()(path1)

    path2 = layers.DepthwiseConv2D(kernel_size, strides=stride, padding='same', use_bias=use_bias)(input_tensor)
    path2 = layers.BatchNormalization()(path2)
    path2 = layers.ReLU()(path2)

    combined = layers.Concatenate()([path1, path2])
    return combined

def build_dpn_model(input_shape):
    inputs = layers.Input(shape=input_shape)

    x = dpn_block(inputs, 64)
    x = dpn_block(x, 128)
    x = dpn_block(x, 256)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(21, activation='softmax')(x)

    model = keras.models.Model(inputs, outputs)
    return model

input_shape = (224, 224, 3)
dpn_model = build_dpn_model(input_shape)
dpn_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 224, 224, 64)         1728      ['input_1[0][0]']             
                                                                                                  
 depthwise_conv2d (Depthwis  (None, 224, 224, 3)          27        ['input_1[0][0]']             
 eConv2D)                                                                                         
                                                                                                  
 batch_normalization (Batch  (None, 224, 224, 64)         256       ['conv2d[0][0]']          

In [42]:
dpn_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
